# Our Model

In [1]:
import os
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Concatenate, Input, Dropout, Conv2D, SeparableConv2D, BatchNormalization, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Lambda, Reshape, Conv2D, Multiply, LayerNormalization, MultiHeadAttention, Add, Flatten
from tensorflow.keras import initializers
from tensorflow.keras.callbacks import ModelCheckpoint

data_path = "TrainData_3/TrainData_3"
img_size = (224, 224)
batch_size = 8
num_classes = 2


def spatial_attention_module(input_feature):
    # 使用一个1x1的卷积层来生成注意力图
    attention = Conv2D(1, (1, 1), activation='sigmoid')(input_feature)
    # 将注意力图乘以原特征图来增强重要特征
    enhanced_feature = Multiply()([input_feature, attention])
    return enhanced_feature

def build_image_model(input_shape):
    img_input = Input(shape=input_shape, name='image_input')
    img_base_model = EfficientNetB7(include_top=False, input_tensor=img_input, weights='imagenet')
    img_features = img_base_model.output
    img_features = spatial_attention_module(img_features)
    img_features = GlobalAveragePooling2D()(img_features)
    return Model(inputs=img_input, outputs=img_features, name='ImageModel')

def load_images(folder, filenames, color_mode='rgb'):
    images = []
    for filename in filenames:
        if filename.startswith('.'):  # 过滤掉隐藏文件
            continue
        img_path = os.path.join(folder, filename)
        img = load_img(img_path, target_size=img_size, color_mode=color_mode)
        img_array = img_to_array(img)
        images.append(img_array)
    return np.array(images)


def transformer_block(x, num_heads=4):
    # Layer Normalization
    x_norm = LayerNormalization()(x)
    # Multi-head self-attention
    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=64)(x_norm, x_norm)
    # Skip connection
    x = Add()([x, attn_output])
    # Another layer normalization
    x_norm = LayerNormalization()(x)
    # Feed-forward network
    ff_output = Dense(units=512, activation='relu')(x_norm)
    # Final skip connection
    output = Add()([x, ff_output])
    return output

def prepare_dataset(data_path):
    landslide_path = os.path.join(data_path, "landslide")
    non_landslide_path = os.path.join(data_path, "non-landslide")
    
    landslide_filenames = [filename for filename in os.listdir(os.path.join(landslide_path, "image")) if not filename.startswith('.')]
    non_landslide_filenames = [filename for filename in os.listdir(os.path.join(non_landslide_path, "image")) if not filename.startswith('.')]

    landslide_images = load_images(os.path.join(landslide_path, "image"), landslide_filenames)
    landslide_dems = load_images(os.path.join(landslide_path, "dem"), landslide_filenames, color_mode='grayscale')
    non_landslide_images = load_images(os.path.join(non_landslide_path, "image"), non_landslide_filenames)
    non_landslide_dems = load_images(os.path.join(non_landslide_path, "dem"), non_landslide_filenames, color_mode='grayscale')

    landslide_labels = np.ones(len(landslide_filenames), dtype=int)
    non_landslide_labels = np.zeros(len(non_landslide_filenames), dtype=int)

    images = np.concatenate([landslide_images, non_landslide_images], axis=0)
    dems = np.concatenate([landslide_dems, non_landslide_dems], axis=0)
    labels = np.concatenate([landslide_labels, non_landslide_labels], axis=0)

    return images, dems, labels


images, dems, labels = prepare_dataset(data_path)
labels = to_categorical(labels, num_classes=num_classes)

x_img_train, x_img_val, x_dem_train, x_dem_val, y_train, y_val = train_test_split(images, dems, labels, test_size=0.4, random_state=42)

def build_dem_cnn_model(input_shape):
    inputs = Input(shape=input_shape, name='dem_input')
    x = SeparableConv2D(128, (3, 3), activation='relu', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = GlobalAveragePooling2D()(x)
    x = Reshape((1, -1))(x)
    x = transformer_block(x)
    x = Flatten()(x)
    return Model(inputs=inputs, outputs=x, name='Advanced_DEM_CNN')


def build_vae_feature_merger(input_dim):
    input_layer = Input(shape=(input_dim,))
    x = Dense(128, activation='relu')(input_layer)
    z_mean = Dense(64)(x)
    z_log_var = Dense(64)(x)
    
    def sampling(args):
        z_mean, z_log_var = args
        batch = K.shape(z_mean)[0]
        dim = K.int_shape(z_mean)[1]
        epsilon = K.random_normal(shape=(batch, dim))
        return z_mean + K.exp(0.5 * z_log_var) * epsilon

    z = Lambda(sampling)([z_mean, z_log_var])
    encoder = Model(input_layer, [z_mean, z_log_var, z], name='vae_merger')
    return encoder



# Build the dual input model
def build_dual_input_model(input_shape):
    img_model = build_image_model(input_shape)
    img_features = img_model.output

    dem_input_shape = (img_size[0], img_size[1], 1)  
    dem_model = build_dem_cnn_model(dem_input_shape)
    dem_features = dem_model.output

    merged_features = Concatenate()([img_features, dem_features])
    vae_encoder = build_vae_feature_merger(merged_features.shape[1])
    z_mean, z_log_var, z = vae_encoder(merged_features)

    x = Dense(256, activation='relu')(z)
    x = Dropout(0.5)(x)
    output = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=[img_model.input, dem_model.input], outputs=output, name='DualInputModel')
    return model

# Compile the model
model = build_dual_input_model(input_shape=(224, 224, 3))
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Define the checkpoint filepath
checkpoint_filepath = 'best_model.h5'

# Define the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    save_weights_only=False,
    verbose=1
)

# Train the model with the ModelCheckpoint callback
history = model.fit(
    [x_img_train, x_dem_train],
    y_train,
    validation_data=([x_img_val, x_dem_val], y_val),
    batch_size=batch_size,
    epochs=1,
)


y_pred_probs = model.predict([x_img_val, x_dem_val])
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_val, axis=1)

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=['non-landslide', 'landslide']))
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


2024-05-19 02:26:45.351377: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-19 02:27:16.835735: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-19 02:27:17.242966: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46662 MB memory:  -> device: 0, name: NVIDIA L20, pci bus id: 0000:75:01.0, compute capability: 8.9
2024-05-19 02:27:50.261016: I tensorflow/strea

193/193 [==============================] - 36s 157ms/step
Classification Report:
               precision    recall  f1-score   support

non-landslide       0.98      0.89      0.93      4434
    landslide       0.77      0.96      0.86      1733

     accuracy                           0.91      6167
    macro avg       0.88      0.92      0.89      6167
 weighted avg       0.92      0.91      0.91      6167

Accuracy: 0.9090
Precision: 0.9237
Recall: 0.9090
F1 Score: 0.9117


In [1]:
import os
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Concatenate, Input, Dropout, Conv2D, SeparableConv2D, BatchNormalization, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Lambda, Reshape, Conv2D, Multiply, LayerNormalization, MultiHeadAttention, Add, Flatten
from tensorflow.keras import initializers
from tensorflow.keras.callbacks import ModelCheckpoint

data_path = "TrainData_3/TrainData_3"
img_size = (224, 224)
batch_size = 8
num_classes = 2


def spatial_attention_module(input_feature):
    # 使用一个1x1的卷积层来生成注意力图
    attention = Conv2D(1, (1, 1), activation='sigmoid')(input_feature)
    # 将注意力图乘以原特征图来增强重要特征
    enhanced_feature = Multiply()([input_feature, attention])
    return enhanced_feature

def build_image_model(input_shape):
    img_input = Input(shape=input_shape, name='image_input')
    img_base_model = EfficientNetB7(include_top=False, input_tensor=img_input, weights='imagenet')
    img_features = img_base_model.output
    img_features = spatial_attention_module(img_features)
    img_features = GlobalAveragePooling2D()(img_features)
    return Model(inputs=img_input, outputs=img_features, name='ImageModel')

def load_images(folder, filenames, color_mode='rgb'):
    images = []
    for filename in filenames:
        if filename.startswith('.'):  # 过滤掉隐藏文件
            continue
        img_path = os.path.join(folder, filename)
        img = load_img(img_path, target_size=img_size, color_mode=color_mode)
        img_array = img_to_array(img)
        images.append(img_array)
    return np.array(images)


def transformer_block(x, num_heads=4):
    # Layer Normalization
    x_norm = LayerNormalization()(x)
    # Multi-head self-attention
    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=64)(x_norm, x_norm)
    # Skip connection
    x = Add()([x, attn_output])
    # Another layer normalization
    x_norm = LayerNormalization()(x)
    # Feed-forward network
    ff_output = Dense(units=512, activation='relu')(x_norm)
    # Final skip connection
    output = Add()([x, ff_output])
    return output

def prepare_dataset(data_path):
    landslide_path = os.path.join(data_path, "landslide")
    non_landslide_path = os.path.join(data_path, "non-landslide")
    
    landslide_filenames = [filename for filename in os.listdir(os.path.join(landslide_path, "image")) if not filename.startswith('.')]
    non_landslide_filenames = [filename for filename in os.listdir(os.path.join(non_landslide_path, "image")) if not filename.startswith('.')]

    landslide_images = load_images(os.path.join(landslide_path, "image"), landslide_filenames)
    landslide_dems = load_images(os.path.join(landslide_path, "dem"), landslide_filenames, color_mode='grayscale')
    non_landslide_images = load_images(os.path.join(non_landslide_path, "image"), non_landslide_filenames)
    non_landslide_dems = load_images(os.path.join(non_landslide_path, "dem"), non_landslide_filenames, color_mode='grayscale')

    landslide_labels = np.ones(len(landslide_filenames), dtype=int)
    non_landslide_labels = np.zeros(len(non_landslide_filenames), dtype=int)

    images = np.concatenate([landslide_images, non_landslide_images], axis=0)
    dems = np.concatenate([landslide_dems, non_landslide_dems], axis=0)
    labels = np.concatenate([landslide_labels, non_landslide_labels], axis=0)

    return images, dems, labels


images, dems, labels = prepare_dataset(data_path)
labels = to_categorical(labels, num_classes=num_classes)

x_img_train, x_img_val, x_dem_train, x_dem_val, y_train, y_val = train_test_split(images, dems, labels, test_size=0.4, random_state=42)

def build_dem_cnn_model(input_shape):
    inputs = Input(shape=input_shape, name='dem_input')
    x = SeparableConv2D(128, (3, 3), activation='relu', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = GlobalAveragePooling2D()(x)
    x = Reshape((1, -1))(x)
    x = transformer_block(x)
    x = Flatten()(x)
    return Model(inputs=inputs, outputs=x, name='Advanced_DEM_CNN')


def build_vae_feature_merger(input_dim):
    input_layer = Input(shape=(input_dim,))
    x = Dense(128, activation='relu')(input_layer)
    z_mean = Dense(64)(x)
    z_log_var = Dense(64)(x)
    
    def sampling(args):
        z_mean, z_log_var = args
        batch = K.shape(z_mean)[0]
        dim = K.int_shape(z_mean)[1]
        epsilon = K.random_normal(shape=(batch, dim))
        return z_mean + K.exp(0.5 * z_log_var) * epsilon

    z = Lambda(sampling)([z_mean, z_log_var])
    encoder = Model(input_layer, [z_mean, z_log_var, z], name='vae_merger')
    return encoder



# Build the dual input model
def build_dual_input_model(input_shape):
    img_model = build_image_model(input_shape)
    img_features = img_model.output

    dem_input_shape = (img_size[0], img_size[1], 1)  
    dem_model = build_dem_cnn_model(dem_input_shape)
    dem_features = dem_model.output

    merged_features = Concatenate()([img_features, dem_features])
    vae_encoder = build_vae_feature_merger(merged_features.shape[1])
    z_mean, z_log_var, z = vae_encoder(merged_features)

    x = Dense(256, activation='relu')(z)
    x = Dropout(0.5)(x)
    output = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=[img_model.input, dem_model.input], outputs=output, name='DualInputModel')
    return model

# Compile the model
model = build_dual_input_model(input_shape=(224, 224, 3))
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Define the checkpoint filepath
checkpoint_filepath = 'best_model.h5'

# Define the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    save_weights_only=False,
    verbose=1
)

# Train the model with the ModelCheckpoint callback
history = model.fit(
    [x_img_train, x_dem_train],
    y_train,
    validation_data=([x_img_val, x_dem_val], y_val),
    batch_size=batch_size,
    epochs=20,
)


y_pred_probs = model.predict([x_img_val, x_dem_val])
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_val, axis=1)

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=['non-landslide', 'landslide']))
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


2024-05-20 08:40:17.741172: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-20 08:40:50.110337: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-20 08:40:50.543555: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46662 MB memory:  -> device: 0, name: NVIDIA L20, pci bus id: 0000:65:01.0, compute capability: 8.9


Epoch 1/20


2024-05-20 08:41:24.810777: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101
2024-05-20 08:41:25.540421: W tensorflow/stream_executor/gpu/asm_compiler.cc:230] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.9
2024-05-20 08:41:25.540441: W tensorflow/stream_executor/gpu/asm_compiler.cc:233] Used ptxas at ptxas
2024-05-20 08:41:25.540487: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2024-05-20 08:41:26.211057: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1157/1157 [==============================] - 405s 328ms/step - loss: 0.3534 - accuracy: 0.8474 - val_loss: 1.1830 - val_accuracy: 0.7540
Epoch 2/20
1157/1157 [==============================] - 369s 319ms/step - loss: 0.2071 - accuracy: 0.9203 - val_loss: 0.1662 - val_accuracy: 0.9330
Epoch 3/20
1157/1157 [==============================] - 369s 319ms/step - loss: 0.1224 - accuracy: 0.9541 - val_loss: 0.1548 - val_accuracy: 0.9376
Epoch 4/20
1157/1157 [==============================] - 369s 319ms/step - loss: 0.0836 - accuracy: 0.9718 - val_loss: 0.2102 - val_accuracy: 0.9335
Epoch 5/20
1157/1157 [==============================] - 369s 319ms/step - loss: 0.0650 - accuracy: 0.9768 - val_loss: 0.1998 - val_accuracy: 0.9338
Epoch 6/20
1157/1157 [==============================] - 370s 320ms/step - loss: 0.0500 - accuracy: 0.9837 - val_loss: 0.1868 - val_accuracy: 0.9400
Epoch 7/20
1157/1157 [==============================] - 369s 319ms/step - loss: 0.0377 - accuracy: 0.9884 - val_loss: 0.240

## Other Model

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Concatenate, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 数据集路径
data_dir = "TrainData_3/TrainData_3"

# 定义数据生成器
datagen = ImageDataGenerator(rescale=1./255)

# 加载数据集
batch_size = 32
image_height = 224
image_width = 224

# 加载landslide数据
landslide_dir = os.path.join(data_dir, "landslide")
landslide_dem_dir = os.path.join(landslide_dir, "dem")
landslide_image_dir = os.path.join(landslide_dir, "image")
landslide_mask_dir = os.path.join(landslide_dir, "mask")

# 加载non-landslide数据
non_landslide_dir = os.path.join(data_dir, "non-landslide")
non_landslide_dem_dir = os.path.join(non_landslide_dir, "dem")
non_landslide_image_dir = os.path.join(non_landslide_dir, "image")

# 读取并预处理数据函数
def read_and_preprocess_data(image_dir, dem_dir=None):
    image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if not filename.startswith('.')]
    images = [tf.keras.preprocessing.image.load_img(img_path, target_size=(image_height, image_width)) for img_path in image_paths]
    images = [tf.keras.preprocessing.image.img_to_array(img) for img in images]
    images = np.array(images)

    if dem_dir:
        dem_paths = [os.path.join(dem_dir, filename) for filename in os.listdir(dem_dir) if not filename.startswith('.')]
        dem_images = [tf.keras.preprocessing.image.load_img(dem_path, target_size=(image_height, image_width)) for dem_path in dem_paths]
        dem_images = [tf.keras.preprocessing.image.img_to_array(dem) for dem in dem_images]
        dem_images = np.array(dem_images)
        return images, dem_images
    else:
        return images

# 读取并预处理landslide数据
landslide_images, landslide_dem_images = read_and_preprocess_data(landslide_image_dir, landslide_dem_dir)
landslide_labels = np.ones(len(landslide_images))

# 读取并预处理non-landslide数据
non_landslide_images = read_and_preprocess_data(non_landslide_image_dir)
non_landslide_labels = np.zeros(len(non_landslide_images))

# 合并数据集
images = np.concatenate((landslide_images, non_landslide_images), axis=0)
dem_images = np.concatenate((landslide_dem_images, np.zeros_like(non_landslide_images)), axis=0)  # 对于non-landslide数据，DEM图像全为0
labels = np.concatenate((landslide_labels, non_landslide_labels), axis=0)

# 切分数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# 构建图像处理模型（使用ResNet50）
input_image = Input(shape=(image_height, image_width, 3))
base_model_image = ResNet50(include_top=False, weights='imagenet', input_tensor=input_image)
image_features = GlobalAveragePooling2D()(base_model_image.output)
image_output = Dense(256, activation='relu')(image_features)

# 构建DEM处理模型（使用简单的CNN模型）
input_dem = Input(shape=(image_height, image_width, 3))
dem_conv1 = Conv2D(32, kernel_size=(3, 3), activation='relu')(input_dem)
dem_pool1 = MaxPooling2D(pool_size=(2, 2))(dem_conv1)
dem_conv2 = Conv2D(64, kernel_size=(3, 3), activation='relu')(dem_pool1)
dem_pool2 = MaxPooling2D(pool_size=(2, 2))(dem_conv2)
dem_flat = Flatten()(dem_pool2)
dem_output = Dense(256, activation='relu')(dem_flat)

# 将两个模型输出连接
merged = Concatenate()([image_output, dem_output])

# 最终的分类层
output = Dense(1, activation='sigmoid')(merged)

# 定义模型
model = Model(inputs=[input_image, input_dem], outputs=output)

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit([X_train, dem_images[:len(X_train)]], y_train, batch_size=batch_size, epochs=50, validation_split=0.2)

# 评估模型
test_loss, test_acc = model.evaluate([X_test, dem_images[len(X_train):]], y_test, verbose=2)
print('\nTest accuracy:', test_acc)

# 模型预测
y_pred = model.predict([X_test, dem_images[len(X_train):]])
y_pred_classes = np.round(y_pred)

# 输出性能报告结果
report = classification_report(y_test, y_pred_classes, target_names=['non-landslide', 'landslide'])
print("Classification Report:\n", report)

# 输出其他性能指标
accuracy = np.mean(y_test == y_pred_classes)
precision = np.mean(y_pred_classes[y_test == 1] == 1)
recall = np.mean(y_pred_classes[y_test == 1] == y_test[y_test == 1])
f1_score = 2 * precision * recall / (precision + recall)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)




94765736/94765736 [==============================] - 5s 0us/step
Epoch 1/50
309/309 [==============================] - 38s 97ms/step - loss: 11.3002 - accuracy: 0.8763 - val_loss: 0.3572 - val_accuracy: 0.8342
Epoch 2/50
309/309 [==============================] - 26s 84ms/step - loss: 0.2327 - accuracy: 0.9123 - val_loss: 0.2395 - val_accuracy: 0.8954
Epoch 3/50
309/309 [==============================] - 25s 82ms/step - loss: 0.2560 - accuracy: 0.9102 - val_loss: 0.2226 - val_accuracy: 0.8983
Epoch 4/50
309/309 [==============================] - 24s 78ms/step - loss: 0.1928 - accuracy: 0.9226 - val_loss: 0.5469 - val_accuracy: 0.7921
Epoch 5/50
309/309 [==============================] - 25s 81ms/step - loss: 0.1738 - accuracy: 0.9287 - val_loss: 0.8562 - val_accuracy: 0.7110
Epoch 6/50
309/309 [==============================] - 24s 76ms/step - loss: 0.1684 - accuracy: 0.9319 - val_loss: 0.2431 - val_accuracy: 0.9031
Epoch 7/50
309/309 [==============================] - 23s 74ms/step - 

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, InceptionV3
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Concatenate, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 数据集路径
data_dir = "TrainData_3/TrainData_3"

# 定义数据生成器
datagen = ImageDataGenerator(rescale=1./255)

# 加载数据集
batch_size = 32
image_height = 224
image_width = 224

# 加载landslide数据
landslide_dir = os.path.join(data_dir, "landslide")
landslide_dem_dir = os.path.join(landslide_dir, "dem")
landslide_image_dir = os.path.join(landslide_dir, "image")
landslide_mask_dir = os.path.join(landslide_dir, "mask")

# 加载non-landslide数据
non_landslide_dir = os.path.join(data_dir, "non-landslide")
non_landslide_dem_dir = os.path.join(non_landslide_dir, "dem")
non_landslide_image_dir = os.path.join(non_landslide_dir, "image")

# 读取并预处理数据函数
def read_and_preprocess_data(image_dir, dem_dir=None):
    image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if not filename.startswith('.')]
    images = [tf.keras.preprocessing.image.load_img(img_path, target_size=(image_height, image_width)) for img_path in image_paths]
    images = [tf.keras.preprocessing.image.img_to_array(img) for img in images]
    images = np.array(images)

    if dem_dir:
        dem_paths = [os.path.join(dem_dir, filename) for filename in os.listdir(dem_dir) if not filename.startswith('.')]
        dem_images = [tf.keras.preprocessing.image.load_img(dem_path, target_size=(image_height, image_width)) for dem_path in dem_paths]
        dem_images = [tf.keras.preprocessing.image.img_to_array(dem) for dem in dem_images]
        dem_images = np.array(dem_images)
        return images, dem_images
    else:
        return images

# 读取并预处理landslide数据
landslide_images, landslide_dem_images = read_and_preprocess_data(landslide_image_dir, landslide_dem_dir)
landslide_labels = np.ones(len(landslide_images))

# 读取并预处理non-landslide数据
non_landslide_images = read_and_preprocess_data(non_landslide_image_dir)
non_landslide_labels = np.zeros(len(non_landslide_images))

# 合并数据集
images = np.concatenate((landslide_images, non_landslide_images), axis=0)
dem_images = np.concatenate((landslide_dem_images, np.zeros_like(non_landslide_images)), axis=0)  # 对于non-landslide数据，DEM图像全为0
labels = np.concatenate((landslide_labels, non_landslide_labels), axis=0)

# 切分数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# 构建图像处理模型（使用VGG16）
input_image = Input(shape=(image_height, image_width, 3))
base_model_image = VGG16(include_top=False, weights='imagenet', input_tensor=input_image)
image_features = GlobalAveragePooling2D()(base_model_image.output)
image_output = Dense(256, activation='relu')(image_features)

# 构建DEM处理模型（使用InceptionV3）
input_dem = Input(shape=(image_height, image_width, 3))
dem_features = InceptionV3(include_top=False, weights='imagenet', input_tensor=input_dem)(input_dem)
dem_features = GlobalAveragePooling2D()(dem_features)
dem_output = Dense(256, activation='relu')(dem_features)

# 将两个模型输出连接
merged = Concatenate()([image_output, dem_output])

# 最终的分类层
output = Dense(1, activation='sigmoid')(merged)

# 定义模型
model = Model(inputs=[input_image, input_dem], outputs=output)

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit([X_train, dem_images[:len(X_train)]], y_train, batch_size=batch_size, epochs=50, validation_split=0.2)

# 评估模型
test_loss, test_acc = model.evaluate([X_test, dem_images[len(X_train):]], y_test, verbose=2)
print('\nTest accuracy:', test_acc)

# 模型预测
y_pred = model.predict([X_test, dem_images[len(X_train):]])
y_pred_classes = np.round(y_pred)

# 输出性能报告结果
report = classification_report(y_test, y_pred_classes, target_names=['non-landslide', 'landslide'])
print("Classification Report:\n", report)

# 输出其他性能指标
accuracy = np.mean(y_test == y_pred_classes)
precision = np.mean(y_pred_classes[y_test == 1] == 1)
recall = np.mean(y_pred_classes[y_test == 1] == y_test[y_test == 1])
f1_score = 2 * precision * recall / (precision + recall)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


87910968/87910968 [==============================] - 5s 0us/step
Epoch 1/50
309/309 [==============================] - 57s 153ms/step - loss: 2.3900 - accuracy: 0.8132 - val_loss: 0.3109 - val_accuracy: 0.8573
Epoch 2/50
309/309 [==============================] - 42s 135ms/step - loss: 0.2918 - accuracy: 0.8702 - val_loss: 4.8118 - val_accuracy: 0.7175
Epoch 3/50
309/309 [==============================] - 42s 135ms/step - loss: 0.2692 - accuracy: 0.8860 - val_loss: 0.3038 - val_accuracy: 0.8553
Epoch 4/50
309/309 [==============================] - 42s 135ms/step - loss: 0.2478 - accuracy: 0.8953 - val_loss: 0.2296 - val_accuracy: 0.9072
Epoch 5/50
309/309 [==============================] - 42s 135ms/step - loss: 0.2332 - accuracy: 0.8974 - val_loss: 0.2422 - val_accuracy: 0.8881
Epoch 6/50
309/309 [==============================] - 40s 130ms/step - loss: 0.2272 - accuracy: 0.9020 - val_loss: 0.2299 - val_accuracy: 0.8999
Epoch 7/50
309/309 [==============================] - 41s 133ms/s

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, MobileNetV2
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 数据集路径
data_dir = "TrainData_3/TrainData_3"

# 定义数据生成器
datagen = ImageDataGenerator(rescale=1./255)

# 加载数据集
batch_size = 32
image_height = 224
image_width = 224

# 加载landslide数据
landslide_dir = os.path.join(data_dir, "landslide")
landslide_dem_dir = os.path.join(landslide_dir, "dem")
landslide_image_dir = os.path.join(landslide_dir, "image")
landslide_mask_dir = os.path.join(landslide_dir, "mask")

# 加载non-landslide数据
non_landslide_dir = os.path.join(data_dir, "non-landslide")
non_landslide_dem_dir = os.path.join(non_landslide_dir, "dem")
non_landslide_image_dir = os.path.join(non_landslide_dir, "image")

# 读取并预处理数据函数
def read_and_preprocess_data(image_dir, dem_dir=None):
    image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if not filename.startswith('.')]
    images = [tf.keras.preprocessing.image.load_img(img_path, target_size=(image_height, image_width)) for img_path in image_paths]
    images = [tf.keras.preprocessing.image.img_to_array(img) for img in images]
    images = np.array(images)

    if dem_dir:
        dem_paths = [os.path.join(dem_dir, filename) for filename in os.listdir(dem_dir) if not filename.startswith('.')]
        dem_images = [tf.keras.preprocessing.image.load_img(dem_path, target_size=(image_height, image_width)) for dem_path in dem_paths]
        dem_images = [tf.keras.preprocessing.image.img_to_array(dem) for dem in dem_images]
        dem_images = np.array(dem_images)
        return images, dem_images
    else:
        return images

# 读取并预处理landslide数据
landslide_images, landslide_dem_images = read_and_preprocess_data(landslide_image_dir, landslide_dem_dir)
landslide_labels = np.ones(len(landslide_images))

# 读取并预处理non-landslide数据
non_landslide_images = read_and_preprocess_data(non_landslide_image_dir)
non_landslide_labels = np.zeros(len(non_landslide_images))

# 合并数据集
images = np.concatenate((landslide_images, non_landslide_images), axis=0)
dem_images = np.concatenate((landslide_dem_images, np.zeros_like(non_landslide_images)), axis=0)  # 对于non-landslide数据，DEM图像全为0
labels = np.concatenate((landslide_labels, non_landslide_labels), axis=0)

# 切分数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# 构建图像处理模型（使用ResNet50）
input_image = Input(shape=(image_height, image_width, 3))
base_model_image = ResNet50(include_top=False, weights='imagenet', input_tensor=input_image)
image_features = GlobalAveragePooling2D()(base_model_image.output)
image_output = Dense(256, activation='relu')(image_features)

# 构建DEM处理模型（使用MobileNetV2）
input_dem = Input(shape=(image_height, image_width, 3))
dem_features = MobileNetV2(include_top=False, weights='imagenet', input_tensor=input_dem)(input_dem)
dem_features = GlobalAveragePooling2D()(dem_features)
dem_output = Dense(256, activation='relu')(dem_features)

# 将两个模型输出连接
merged = Concatenate()([image_output, dem_output])

# 最终的分类层
output = Dense(1, activation='sigmoid')(merged)

# 定义模型
model = Model(inputs=[input_image, input_dem], outputs=output)

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit([X_train, dem_images[:len(X_train)]], y_train, batch_size=batch_size, epochs=50, validation_split=0.2)

# 评估模型
test_loss, test_acc = model.evaluate([X_test, dem_images[len(X_train):]], y_test, verbose=2)
print('\nTest accuracy:', test_acc)

# 模型预测
y_pred = model.predict([X_test, dem_images[len(X_train):]])
y_pred_classes = np.round(y_pred)

# 输出性能报告结果
report = classification_report(y_test, y_pred_classes, target_names=['non-landslide', 'landslide'])
print("Classification Report:\n", report)

# 输出其他性能指标
accuracy = np.mean(y_test == y_pred_classes)
precision = np.mean(y_pred_classes[y_test == 1] == 1)
recall = np.mean(y_pred_classes[y_test == 1] == y_test[y_test == 1])
f1_score = 2 * precision * recall / (precision + recall)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


2024-05-19 09:31:23.265574: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-19 09:32:00.046495: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-19 09:32:00.539340: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46662 MB memory:  -> device: 0, name: NVIDIA L20, pci bus id: 0000:65:01.0, compute capability: 8.9


9406464/9406464 [==============================] - 1s 0us/step
Epoch 1/50


2024-05-19 09:32:23.595578: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101
2024-05-19 09:32:24.284074: W tensorflow/stream_executor/gpu/asm_compiler.cc:230] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.9
2024-05-19 09:32:24.284093: W tensorflow/stream_executor/gpu/asm_compiler.cc:233] Used ptxas at ptxas
2024-05-19 09:32:24.284144: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2024-05-19 09:32:25.087762: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


309/309 [==============================] - 61s 161ms/step - loss: 0.2867 - accuracy: 0.8784 - val_loss: 1.3652 - val_accuracy: 0.8245
Epoch 2/50
309/309 [==============================] - 48s 156ms/step - loss: 0.2176 - accuracy: 0.9070 - val_loss: 0.6942 - val_accuracy: 0.8553
Epoch 3/50
309/309 [==============================] - 48s 156ms/step - loss: 0.2081 - accuracy: 0.9119 - val_loss: 0.2233 - val_accuracy: 0.9104
Epoch 4/50
309/309 [==============================] - 47s 152ms/step - loss: 0.1922 - accuracy: 0.9222 - val_loss: 0.2421 - val_accuracy: 0.8970
Epoch 5/50
309/309 [==============================] - 48s 154ms/step - loss: 0.1811 - accuracy: 0.9256 - val_loss: 0.2313 - val_accuracy: 0.8946
Epoch 6/50
309/309 [==============================] - 48s 155ms/step - loss: 0.1788 - accuracy: 0.9241 - val_loss: 0.5734 - val_accuracy: 0.8180
Epoch 7/50
309/309 [==============================] - 48s 155ms/step - loss: 0.1691 - accuracy: 0.9276 - val_loss: 0.2839 - val_accuracy: 0.8

In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2, Xception
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 数据集路径
data_dir = "TrainData_3/TrainData_3"

# 定义数据生成器
datagen = ImageDataGenerator(rescale=1./255)

# 加载数据集
batch_size = 32
image_height = 224
image_width = 224

# 加载landslide数据
landslide_dir = os.path.join(data_dir, "landslide")
landslide_dem_dir = os.path.join(landslide_dir, "dem")
landslide_image_dir = os.path.join(landslide_dir, "image")
landslide_mask_dir = os.path.join(landslide_dir, "mask")

# 加载non-landslide数据
non_landslide_dir = os.path.join(data_dir, "non-landslide")
non_landslide_dem_dir = os.path.join(non_landslide_dir, "dem")
non_landslide_image_dir = os.path.join(non_landslide_dir, "image")

# 读取并预处理数据函数
def read_and_preprocess_data(image_dir, dem_dir=None):
    image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if not filename.startswith('.')]
    images = [tf.keras.preprocessing.image.load_img(img_path, target_size=(image_height, image_width)) for img_path in image_paths]
    images = [tf.keras.preprocessing.image.img_to_array(img) for img in images]
    images = np.array(images)

    if dem_dir:
        dem_paths = [os.path.join(dem_dir, filename) for filename in os.listdir(dem_dir) if not filename.startswith('.')]
        dem_images = [tf.keras.preprocessing.image.load_img(dem_path, target_size=(image_height, image_width)) for dem_path in dem_paths]
        dem_images = [tf.keras.preprocessing.image.img_to_array(dem) for dem in dem_images]
        dem_images = np.array(dem_images)
        return images, dem_images
    else:
        return images

# 读取并预处理landslide数据
landslide_images, landslide_dem_images = read_and_preprocess_data(landslide_image_dir, landslide_dem_dir)
landslide_labels = np.ones(len(landslide_images))

# 读取并预处理non-landslide数据
non_landslide_images = read_and_preprocess_data(non_landslide_image_dir)
non_landslide_labels = np.zeros(len(non_landslide_images))

# 合并数据集
images = np.concatenate((landslide_images, non_landslide_images), axis=0)
dem_images = np.concatenate((landslide_dem_images, np.zeros_like(non_landslide_images)), axis=0)  # 对于non-landslide数据，DEM图像全为0
labels = np.concatenate((landslide_labels, non_landslide_labels), axis=0)

# 切分数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# 构建图像处理模型（使用MobileNetV2）
input_image = Input(shape=(image_height, image_width, 3))
base_model_image = MobileNetV2(include_top=False, weights='imagenet', input_tensor=input_image)
image_features = GlobalAveragePooling2D()(base_model_image.output)
image_output = Dense(256, activation='relu')(image_features)

# 构建DEM处理模型（使用Xception）
input_dem = Input(shape=(image_height, image_width, 3))
dem_features = Xception(include_top=False, weights='imagenet', input_tensor=input_dem)(input_dem)
dem_features = GlobalAveragePooling2D()(dem_features)
dem_output = Dense(256, activation='relu')(dem_features)

# 将两个模型输出连接
merged = Concatenate()([image_output, dem_output])

# 最终的分类层
output = Dense(1, activation='sigmoid')(merged)

# 定义模型
model = Model(inputs=[input_image, input_dem], outputs=output)

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit([X_train, dem_images[:len(X_train)]], y_train, batch_size=batch_size, epochs=50, validation_split=0.2)

# 评估模型
test_loss, test_acc = model.evaluate([X_test, dem_images[len(X_train):]], y_test, verbose=2)
print('\nTest accuracy:', test_acc)

# 模型预测
y_pred = model.predict([X_test, dem_images[len(X_train):]])
y_pred_classes = np.round(y_pred)

# 输出性能报告结果
report = classification_report(y_test, y_pred_classes, target_names=['non-landslide', 'landslide'])
print("Classification Report:\n", report)

# 输出其他性能指标
accuracy = np.mean(y_test == y_pred_classes)
precision = np.mean(y_pred_classes[y_test == 1] == 1)
recall = np.mean(y_pred_classes[y_test == 1] == y_test[y_test == 1])
f1_score = 2 * precision * recall / (precision + recall)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


83683744/83683744 [==============================] - 5s 0us/step
Epoch 1/50
309/309 [==============================] - 94s 274ms/step - loss: 0.2638 - accuracy: 0.8889 - val_loss: 2.6321 - val_accuracy: 0.6019
Epoch 2/50
309/309 [==============================] - 82s 266ms/step - loss: 0.2156 - accuracy: 0.9086 - val_loss: 3.3888 - val_accuracy: 0.4795
Epoch 3/50
309/309 [==============================] - 82s 267ms/step - loss: 0.1952 - accuracy: 0.9150 - val_loss: 2.4905 - val_accuracy: 0.5517
Epoch 4/50
309/309 [==============================] - 83s 268ms/step - loss: 0.1724 - accuracy: 0.9320 - val_loss: 0.3643 - val_accuracy: 0.8950
Epoch 5/50
309/309 [==============================] - 81s 262ms/step - loss: 0.1517 - accuracy: 0.9378 - val_loss: 3.3872 - val_accuracy: 0.3518
Epoch 6/50
309/309 [==============================] - 81s 261ms/step - loss: 0.1539 - accuracy: 0.9356 - val_loss: 1.8792 - val_accuracy: 0.5987
Epoch 7/50
309/309 [==============================] - 81s 262ms/s

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet, DenseNet121
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 数据集路径
data_dir = "TrainData_3/TrainData_3"

# 定义数据生成器
datagen = ImageDataGenerator(rescale=1./255)

# 加载数据集
batch_size = 32
image_height = 224
image_width = 224

# 加载landslide数据
landslide_dir = os.path.join(data_dir, "landslide")
landslide_dem_dir = os.path.join(landslide_dir, "dem")
landslide_image_dir = os.path.join(landslide_dir, "image")
landslide_mask_dir = os.path.join(landslide_dir, "mask")

# 加载non-landslide数据
non_landslide_dir = os.path.join(data_dir, "non-landslide")
non_landslide_dem_dir = os.path.join(non_landslide_dir, "dem")
non_landslide_image_dir = os.path.join(non_landslide_dir, "image")

# 读取并预处理数据函数
def read_and_preprocess_data(image_dir, dem_dir=None):
    image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if not filename.startswith('.')]
    images = [tf.keras.preprocessing.image.load_img(img_path, target_size=(image_height, image_width)) for img_path in image_paths]
    images = [tf.keras.preprocessing.image.img_to_array(img) for img in images]
    images = np.array(images)

    if dem_dir:
        dem_paths = [os.path.join(dem_dir, filename) for filename in os.listdir(dem_dir) if not filename.startswith('.')]
        dem_images = [tf.keras.preprocessing.image.load_img(dem_path, target_size=(image_height, image_width)) for dem_path in dem_paths]
        dem_images = [tf.keras.preprocessing.image.img_to_array(dem) for dem in dem_images]
        dem_images = np.array(dem_images)
        return images, dem_images
    else:
        return images

# 读取并预处理landslide数据
landslide_images, landslide_dem_images = read_and_preprocess_data(landslide_image_dir, landslide_dem_dir)
landslide_labels = np.ones(len(landslide_images))

# 读取并预处理non-landslide数据
non_landslide_images = read_and_preprocess_data(non_landslide_image_dir)
non_landslide_labels = np.zeros(len(non_landslide_images))

# 合并数据集
images = np.concatenate((landslide_images, non_landslide_images), axis=0)
dem_images = np.concatenate((landslide_dem_images, np.zeros_like(non_landslide_images)), axis=0)  # 对于non-landslide数据，DEM图像全为0
labels = np.concatenate((landslide_labels, non_landslide_labels), axis=0)

# 切分数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# 构建图像处理模型（使用MobileNet）
input_image = Input(shape=(image_height, image_width, 3))
base_model_image = MobileNet(include_top=False, weights='imagenet', input_tensor=input_image)
image_features = GlobalAveragePooling2D()(base_model_image.output)
image_output = Dense(256, activation='relu')(image_features)

# 构建DEM处理模型（使用DenseNet121）
input_dem = Input(shape=(image_height, image_width, 3))
dem_features = DenseNet121(include_top=False, weights='imagenet', input_tensor=input_dem)(input_dem)
dem_features = GlobalAveragePooling2D()(dem_features)
dem_output = Dense(256, activation='relu')(dem_features)

# 将两个模型输出连接
merged = Concatenate()([image_output, dem_output])

# 最终的分类层
output = Dense(1, activation='sigmoid')(merged)

# 定义模型
model = Model(inputs=[input_image, input_dem], outputs=output)

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit([X_train, dem_images[:len(X_train)]], y_train, batch_size=batch_size, epochs=50, validation_split=0.2)

# 评估模型
test_loss, test_acc = model.evaluate([X_test, dem_images[len(X_train):]], y_test, verbose=2)
print('\nTest accuracy:', test_acc)

# 模型预测
y_pred = model.predict([X_test, dem_images[len(X_train):]])
y_pred_classes = np.round(y_pred)

# 输出性能报告结果
report = classification_report(y_test, y_pred_classes, target_names=['non-landslide', 'landslide'])
print("Classification Report:\n", report)

# 输出其他性能指标
accuracy = np.mean(y_test == y_pred_classes)
precision = np.mean(y_pred_classes[y_test == 1] == 1)
recall = np.mean(y_pred_classes[y_test == 1] == y_test[y_test == 1])
f1_score = 2 * precision * recall / (precision + recall)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


2024-05-19 14:25:02.020963: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


2024-05-19 14:25:38.071579: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-19 14:25:38.423847: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46662 MB memory:  -> device: 0, name: NVIDIA L20, pci bus id: 0000:65:01.0, compute capability: 8.9


29084464/29084464 [==============================] - 5s 0us/step
Epoch 1/50


2024-05-19 14:26:17.398273: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101
2024-05-19 14:26:18.088751: W tensorflow/stream_executor/gpu/asm_compiler.cc:230] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.9
2024-05-19 14:26:18.088771: W tensorflow/stream_executor/gpu/asm_compiler.cc:233] Used ptxas at ptxas
2024-05-19 14:26:18.088822: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2024-05-19 14:26:18.803243: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


309/309 [==============================] - 70s 175ms/step - loss: 0.2705 - accuracy: 0.8922 - val_loss: 1.3282 - val_accuracy: 0.6773
Epoch 2/50
309/309 [==============================] - 49s 159ms/step - loss: 0.1935 - accuracy: 0.9223 - val_loss: 0.2277 - val_accuracy: 0.9064
Epoch 3/50
309/309 [==============================] - 47s 153ms/step - loss: 0.1669 - accuracy: 0.9337 - val_loss: 0.3083 - val_accuracy: 0.9019
Epoch 4/50
309/309 [==============================] - 48s 154ms/step - loss: 0.1528 - accuracy: 0.9403 - val_loss: 0.5149 - val_accuracy: 0.8273
Epoch 5/50
309/309 [==============================] - 47s 153ms/step - loss: 0.1326 - accuracy: 0.9484 - val_loss: 0.2159 - val_accuracy: 0.9262
Epoch 6/50
309/309 [==============================] - 47s 154ms/step - loss: 0.1085 - accuracy: 0.9556 - val_loss: 0.2427 - val_accuracy: 0.9092
Epoch 7/50
309/309 [==============================] - 47s 154ms/step - loss: 0.0943 - accuracy: 0.9642 - val_loss: 0.2564 - val_accuracy: 0.9

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2, InceptionResNetV2
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 数据集路径
data_dir = "TrainData_3/TrainData_3"

# 定义数据生成器
datagen = ImageDataGenerator(rescale=1./255)

# 加载数据集
batch_size = 32
image_height = 224
image_width = 224

# 加载landslide数据
landslide_dir = os.path.join(data_dir, "landslide")
landslide_dem_dir = os.path.join(landslide_dir, "dem")
landslide_image_dir = os.path.join(landslide_dir, "image")
landslide_mask_dir = os.path.join(landslide_dir, "mask")

# 加载non-landslide数据
non_landslide_dir = os.path.join(data_dir, "non-landslide")
non_landslide_dem_dir = os.path.join(non_landslide_dir, "dem")
non_landslide_image_dir = os.path.join(non_landslide_dir, "image")

# 读取并预处理数据函数
def read_and_preprocess_data(image_dir, dem_dir=None):
    image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if not filename.startswith('.')]
    images = [tf.keras.preprocessing.image.load_img(img_path, target_size=(image_height, image_width)) for img_path in image_paths]
    images = [tf.keras.preprocessing.image.img_to_array(img) for img in images]
    images = np.array(images)

    if dem_dir:
        dem_paths = [os.path.join(dem_dir, filename) for filename in os.listdir(dem_dir) if not filename.startswith('.')]
        dem_images = [tf.keras.preprocessing.image.load_img(dem_path, target_size=(image_height, image_width)) for dem_path in dem_paths]
        dem_images = [tf.keras.preprocessing.image.img_to_array(dem) for dem in dem_images]
        dem_images = np.array(dem_images)
        return images, dem_images
    else:
        return images

# 读取并预处理landslide数据
landslide_images, landslide_dem_images = read_and_preprocess_data(landslide_image_dir, landslide_dem_dir)
landslide_labels = np.ones(len(landslide_images))

# 读取并预处理non-landslide数据
non_landslide_images = read_and_preprocess_data(non_landslide_image_dir)
non_landslide_labels = np.zeros(len(non_landslide_images))

# 合并数据集
images = np.concatenate((landslide_images, non_landslide_images), axis=0)
dem_images = np.concatenate((landslide_dem_images, np.zeros_like(non_landslide_images)), axis=0)  # 对于non-landslide数据，DEM图像全为0
labels = np.concatenate((landslide_labels, non_landslide_labels), axis=0)

# 切分数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# 构建图像处理模型（使用MobileNetV2）
input_image = Input(shape=(image_height, image_width, 3))
base_model_image = MobileNetV2(include_top=False, weights='imagenet', input_tensor=input_image)
image_features = GlobalAveragePooling2D()(base_model_image.output)
image_output = Dense(256, activation='relu')(image_features)

# 构建DEM处理模型（使用InceptionResNetV2）
input_dem = Input(shape=(image_height, image_width, 3))
dem_features = InceptionResNetV2(include_top=False, weights='imagenet', input_tensor=input_dem)(input_dem)
dem_features = GlobalAveragePooling2D()(dem_features)
dem_output = Dense(256, activation='relu')(dem_features)

# 将两个模型输出连接
merged = Concatenate()([image_output, dem_output])

# 最终的分类层
output = Dense(1, activation='sigmoid')(merged)

# 定义模型
model = Model(inputs=[input_image, input_dem], outputs=output)

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit([X_train, dem_images[:len(X_train)]], y_train, batch_size=batch_size, epochs=50, validation_split=0.2)

# 评估模型
test_loss, test_acc = model.evaluate([X_test, dem_images[len(X_train):]], y_test, verbose=2)
print('\nTest accuracy:', test_acc)

# 模型预测
y_pred = model.predict([X_test, dem_images[len(X_train):]])
y_pred_classes = np.round(y_pred)

# 输出性能报告结果
report = classification_report(y_test, y_pred_classes, target_names=['non-landslide', 'landslide'])
print("Classification Report:\n", report)

# 输出其他性能指标
accuracy = np.mean(y_test == y_pred_classes)
precision = np.mean(y_pred_classes[y_test == 1] == 1)
recall = np.mean(y_pred_classes[y_test == 1] == y_test[y_test == 1])
f1_score = 2 * precision * recall / (precision + recall)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


219055592/219055592 [==============================] - 26s 0us/step
Epoch 1/50
309/309 [==============================] - 83s 210ms/step - loss: 0.2595 - accuracy: 0.8937 - val_loss: 1.9111 - val_accuracy: 0.4706
Epoch 2/50
309/309 [==============================] - 59s 189ms/step - loss: 0.2105 - accuracy: 0.9113 - val_loss: 1.1766 - val_accuracy: 0.7353
Epoch 3/50
309/309 [==============================] - 59s 190ms/step - loss: 0.1807 - accuracy: 0.9246 - val_loss: 2.4834 - val_accuracy: 0.5687
Epoch 4/50
309/309 [==============================] - 58s 189ms/step - loss: 0.1656 - accuracy: 0.9335 - val_loss: 0.5414 - val_accuracy: 0.8687
Epoch 5/50
309/309 [==============================] - 59s 190ms/step - loss: 0.1567 - accuracy: 0.9351 - val_loss: 1.0611 - val_accuracy: 0.8034
Epoch 6/50
309/309 [==============================] - 59s 190ms/step - loss: 0.1434 - accuracy: 0.9415 - val_loss: 0.8022 - val_accuracy: 0.8176
Epoch 7/50
309/309 [==============================] - 60s 193m

In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121, VGG16
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 数据集路径
data_dir = "TrainData_3/TrainData_3"

# 定义数据生成器
datagen = ImageDataGenerator(rescale=1./255)

# 加载数据集
batch_size = 32
image_height = 224
image_width = 224

# 加载landslide数据
landslide_dir = os.path.join(data_dir, "landslide")
landslide_dem_dir = os.path.join(landslide_dir, "dem")
landslide_image_dir = os.path.join(landslide_dir, "image")
landslide_mask_dir = os.path.join(landslide_dir, "mask")

# 加载non-landslide数据
non_landslide_dir = os.path.join(data_dir, "non-landslide")
non_landslide_dem_dir = os.path.join(non_landslide_dir, "dem")
non_landslide_image_dir = os.path.join(non_landslide_dir, "image")

# 读取并预处理数据函数
def read_and_preprocess_data(image_dir, dem_dir=None):
    image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if not filename.startswith('.')]
    images = [tf.keras.preprocessing.image.load_img(img_path, target_size=(image_height, image_width)) for img_path in image_paths]
    images = [tf.keras.preprocessing.image.img_to_array(img) for img in images]
    images = np.array(images)

    if dem_dir:
        dem_paths = [os.path.join(dem_dir, filename) for filename in os.listdir(dem_dir) if not filename.startswith('.')]
        dem_images = [tf.keras.preprocessing.image.load_img(dem_path, target_size=(image_height, image_width)) for dem_path in dem_paths]
        dem_images = [tf.keras.preprocessing.image.img_to_array(dem) for dem in dem_images]
        dem_images = np.array(dem_images)
        return images, dem_images
    else:
        return images

# 读取并预处理landslide数据
landslide_images, landslide_dem_images = read_and_preprocess_data(landslide_image_dir, landslide_dem_dir)
landslide_labels = np.ones(len(landslide_images))

# 读取并预处理non-landslide数据
non_landslide_images = read_and_preprocess_data(non_landslide_image_dir)
non_landslide_labels = np.zeros(len(non_landslide_images))

# 合并数据集
images = np.concatenate((landslide_images, non_landslide_images), axis=0)
dem_images = np.concatenate((landslide_dem_images, np.zeros_like(non_landslide_images)), axis=0)  # 对于non-landslide数据，DEM图像全为0
labels = np.concatenate((landslide_labels, non_landslide_labels), axis=0)

# 切分数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# 构建图像处理模型（使用DenseNet121）
input_image = Input(shape=(image_height, image_width, 3))
base_model_image = DenseNet121(include_top=False, weights='imagenet', input_tensor=input_image)
image_features = GlobalAveragePooling2D()(base_model_image.output)
image_output = Dense(256, activation='relu')(image_features)

# 构建DEM处理模型（使用VGG16）
input_dem = Input(shape=(image_height, image_width, 3))
base_model_dem = VGG16(include_top=False, weights='imagenet', input_tensor=input_dem)
dem_features = GlobalAveragePooling2D()(base_model_dem.output)
dem_output = Dense(256, activation='relu')(dem_features)

# 将两个模型输出连接
merged = Concatenate()([image_output, dem_output])

# 最终的分类层
output = Dense(1, activation='sigmoid')(merged)

# 定义模型
model = Model(inputs=[input_image, input_dem], outputs=output)

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit([X_train, dem_images[:len(X_train)]], y_train, batch_size=batch_size, epochs=50, validation_split=0.2)

# 评估模型
test_loss, test_acc = model.evaluate([X_test, dem_images[len(X_train):]], y_test, verbose=2)
print('\nTest accuracy:', test_acc)

# 模型预测
y_pred = model.predict([X_test, dem_images[len(X_train):]])
y_pred_classes = np.round(y_pred)

# 输出性能报告结果
report = classification_report(y_test, y_pred_classes, target_names=['non-landslide', 'landslide'])
print("Classification Report:\n", report)

# 输出其他性能指标
accuracy = np.mean(y_test == y_pred_classes)
precision = np.mean(y_pred_classes[y_test == 1] == 1)
recall = np.mean(y_pred_classes[y_test == 1] == y_test[y_test == 1])
f1_score = 2 * precision * recall / (precision + recall)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


Epoch 1/50
309/309 [==============================] - 65s 162ms/step - loss: 1.3283 - accuracy: 0.8892 - val_loss: 1.1094 - val_accuracy: 0.8208
Epoch 2/50
309/309 [==============================] - 45s 146ms/step - loss: 0.2157 - accuracy: 0.9100 - val_loss: 0.2217 - val_accuracy: 0.8979
Epoch 3/50
309/309 [==============================] - 47s 152ms/step - loss: 0.1958 - accuracy: 0.9169 - val_loss: 0.3974 - val_accuracy: 0.8606
Epoch 4/50
309/309 [==============================] - 46s 150ms/step - loss: 0.1922 - accuracy: 0.9180 - val_loss: 0.1971 - val_accuracy: 0.9153
Epoch 5/50
309/309 [==============================] - 46s 149ms/step - loss: 0.1816 - accuracy: 0.9240 - val_loss: 0.2187 - val_accuracy: 0.9051
Epoch 6/50
309/309 [==============================] - 46s 150ms/step - loss: 0.1786 - accuracy: 0.9222 - val_loss: 0.2316 - val_accuracy: 0.9007
Epoch 7/50
309/309 [==============================] - 46s 149ms/step - loss: 0.1687 - accuracy: 0.9277 - val_loss: 0.3336 - val_ac

In [4]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121, VGG16
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 数据集路径
data_dir = "TrainData_3/TrainData_3"

# 定义数据生成器
datagen = ImageDataGenerator(rescale=1./255)

# 加载数据集
batch_size = 32
image_height = 224
image_width = 224

# 加载landslide数据
landslide_dir = os.path.join(data_dir, "landslide")
landslide_dem_dir = os.path.join(landslide_dir, "dem")
landslide_image_dir = os.path.join(landslide_dir, "image")
landslide_mask_dir = os.path.join(landslide_dir, "mask")

# 加载non-landslide数据
non_landslide_dir = os.path.join(data_dir, "non-landslide")
non_landslide_dem_dir = os.path.join(non_landslide_dir, "dem")
non_landslide_image_dir = os.path.join(non_landslide_dir, "image")

# 读取并预处理数据函数
def read_and_preprocess_data(image_dir, dem_dir=None):
    image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if not filename.startswith('.')]
    images = [tf.keras.preprocessing.image.load_img(img_path, target_size=(image_height, image_width)) for img_path in image_paths]
    images = [tf.keras.preprocessing.image.img_to_array(img) for img in images]
    images = np.array(images)

    if dem_dir:
        dem_paths = [os.path.join(dem_dir, filename) for filename in os.listdir(dem_dir) if not filename.startswith('.')]
        dem_images = [tf.keras.preprocessing.image.load_img(dem_path, target_size=(image_height, image_width)) for dem_path in dem_paths]
        dem_images = [tf.keras.preprocessing.image.img_to_array(dem) for dem in dem_images]
        dem_images = np.array(dem_images)
        return images, dem_images
    else:
        return images

# 读取并预处理landslide数据
landslide_images, landslide_dem_images = read_and_preprocess_data(landslide_image_dir, landslide_dem_dir)
landslide_labels = np.ones(len(landslide_images))

# 读取并预处理non-landslide数据
non_landslide_images = read_and_preprocess_data(non_landslide_image_dir)
non_landslide_labels = np.zeros(len(non_landslide_images))

# 合并数据集
images = np.concatenate((landslide_images, non_landslide_images), axis=0)
dem_images = np.concatenate((landslide_dem_images, np.zeros_like(non_landslide_images)), axis=0)  # 对于non-landslide数据，DEM图像全为0
labels = np.concatenate((landslide_labels, non_landslide_labels), axis=0)

# 切分数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# 构建图像处理模型（使用DenseNet121）
input_image = Input(shape=(image_height, image_width, 3))
base_model_image = DenseNet121(include_top=False, weights='imagenet', input_tensor=input_image)
image_features = GlobalAveragePooling2D()(base_model_image.output)
image_output = Dense(256, activation='relu')(image_features)

# 构建DEM处理模型（使用VGG16）
input_dem = Input(shape=(image_height, image_width, 3))
base_model_dem = VGG16(include_top=False, weights='imagenet', input_tensor=input_dem)
dem_features = GlobalAveragePooling2D()(base_model_dem.output)
dem_output = Dense(256, activation='relu')(dem_features)

# 将两个模型输出连接
merged = Concatenate()([image_output, dem_output])

# 最终的分类层
output = Dense(1, activation='sigmoid')(merged)

# 定义模型
model = Model(inputs=[input_image, input_dem], outputs=output)

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit([X_train, dem_images[:len(X_train)]], y_train, batch_size=batch_size, epochs=50, validation_split=0.2)

# 评估模型
test_loss, test_acc = model.evaluate([X_test, dem_images[len(X_train):]], y_test, verbose=2)
print('\nTest accuracy:', test_acc)

# 模型预测
y_pred = model.predict([X_test, dem_images[len(X_train):]])
y_pred_classes = np.round(y_pred)

# 输出性能报告结果
report = classification_report(y_test, y_pred_classes, target_names=['non-landslide', 'landslide'])
print("Classification Report:\n", report)

# 输出其他性能指标
accuracy = np.mean(y_test == y_pred_classes)
precision = np.mean(y_pred_classes[y_test == 1] == 1)
recall = np.mean(y_pred_classes[y_test == 1] == y_test[y_test == 1])
f1_score = 2 * precision * recall / (precision + recall)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


Epoch 1/50
309/309 [==============================] - 61s 161ms/step - loss: 0.5835 - accuracy: 0.8935 - val_loss: 0.2597 - val_accuracy: 0.8837
Epoch 2/50
309/309 [==============================] - 48s 156ms/step - loss: 0.2075 - accuracy: 0.9109 - val_loss: 0.1738 - val_accuracy: 0.9274
Epoch 3/50
309/309 [==============================] - 46s 150ms/step - loss: 0.1885 - accuracy: 0.9209 - val_loss: 0.4783 - val_accuracy: 0.8435
Epoch 4/50
309/309 [==============================] - 47s 152ms/step - loss: 0.1862 - accuracy: 0.9213 - val_loss: 0.8120 - val_accuracy: 0.8034
Epoch 5/50
309/309 [==============================] - 46s 150ms/step - loss: 0.1723 - accuracy: 0.9286 - val_loss: 0.2263 - val_accuracy: 0.9177
Epoch 6/50
309/309 [==============================] - 47s 152ms/step - loss: 0.1664 - accuracy: 0.9290 - val_loss: 0.2200 - val_accuracy: 0.9137
Epoch 7/50
309/309 [==============================] - 46s 148ms/step - loss: 0.1574 - accuracy: 0.9368 - val_loss: 0.2532 - val_ac

In [5]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3, MobileNetV2
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 数据集路径
data_dir = "TrainData_3/TrainData_3"

# 定义数据生成器
datagen = ImageDataGenerator(rescale=1./255)

# 加载数据集
batch_size = 32
image_height = 224
image_width = 224

# 加载landslide数据
landslide_dir = os.path.join(data_dir, "landslide")
landslide_dem_dir = os.path.join(landslide_dir, "dem")
landslide_image_dir = os.path.join(landslide_dir, "image")
landslide_mask_dir = os.path.join(landslide_dir, "mask")

# 加载non-landslide数据
non_landslide_dir = os.path.join(data_dir, "non-landslide")
non_landslide_dem_dir = os.path.join(non_landslide_dir, "dem")
non_landslide_image_dir = os.path.join(non_landslide_dir, "image")

# 读取并预处理数据函数
def read_and_preprocess_data(image_dir, dem_dir=None):
    image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if not filename.startswith('.')]
    images = [tf.keras.preprocessing.image.load_img(img_path, target_size=(image_height, image_width)) for img_path in image_paths]
    images = [tf.keras.preprocessing.image.img_to_array(img) for img in images]
    images = np.array(images)

    if dem_dir:
        dem_paths = [os.path.join(dem_dir, filename) for filename in os.listdir(dem_dir) if not filename.startswith('.')]
        dem_images = [tf.keras.preprocessing.image.load_img(dem_path, target_size=(image_height, image_width)) for dem_path in dem_paths]
        dem_images = [tf.keras.preprocessing.image.img_to_array(dem) for dem in dem_images]
        dem_images = np.array(dem_images)
        return images, dem_images
    else:
        return images

# 读取并预处理landslide数据
landslide_images, landslide_dem_images = read_and_preprocess_data(landslide_image_dir, landslide_dem_dir)
landslide_labels = np.ones(len(landslide_images))

# 读取并预处理non-landslide数据
non_landslide_images = read_and_preprocess_data(non_landslide_image_dir)
non_landslide_labels = np.zeros(len(non_landslide_images))

# 合并数据集
images = np.concatenate((landslide_images, non_landslide_images), axis=0)
dem_images = np.concatenate((landslide_dem_images, np.zeros_like(non_landslide_images)), axis=0)  # 对于non-landslide数据，DEM图像全为0
labels = np.concatenate((landslide_labels, non_landslide_labels), axis=0)

# 切分数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# 构建图像处理模型（使用InceptionV3）
input_image = Input(shape=(image_height, image_width, 3))
base_model_image = InceptionV3(include_top=False, weights='imagenet', input_tensor=input_image)
image_features = GlobalAveragePooling2D()(base_model_image.output)
image_output = Dense(256, activation='relu')(image_features)

# 构建DEM处理模型（使用MobileNetV2）
input_dem = Input(shape=(image_height, image_width, 3))
base_model_dem = MobileNetV2(include_top=False, weights='imagenet', input_tensor=input_dem)
dem_features = GlobalAveragePooling2D()(base_model_dem.output)
dem_output = Dense(256, activation='relu')(dem_features)

# 将两个模型输出连接
merged = Concatenate()([image_output, dem_output])

# 最终的分类层
output = Dense(1, activation='sigmoid')(merged)

# 定义模型
model = Model(inputs=[input_image, input_dem], outputs=output)

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit([X_train, dem_images[:len(X_train)]], y_train, batch_size=batch_size, epochs=50, validation_split=0.2)

# 评估模型
test_loss, test_acc = model.evaluate([X_test, dem_images[len(X_train):]], y_test, verbose=2)
print('\nTest accuracy:', test_acc)

# 模型预测
y_pred = model.predict([X_test, dem_images[len(X_train):]])
y_pred_classes = np.round(y_pred)

# 输出性能报告结果
report = classification_report(y_test, y_pred_classes, target_names=['non-landslide', 'landslide'])
print("Classification Report:\n", report)

# 输出其他性能指标
accuracy = np.mean(y_test == y_pred_classes)
precision = np.mean(y_pred_classes[y_test == 1] == 1)
recall = np.mean(y_pred_classes[y_test == 1] == y_test[y_test == 1])
f1_score = 2 * precision * recall / (precision + recall)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


Epoch 1/50
309/309 [==============================] - 61s 151ms/step - loss: 0.2792 - accuracy: 0.8806 - val_loss: 0.5926 - val_accuracy: 0.8472
Epoch 2/50
309/309 [==============================] - 44s 143ms/step - loss: 0.2313 - accuracy: 0.9031 - val_loss: 30.5819 - val_accuracy: 0.7049
Epoch 3/50
309/309 [==============================] - 43s 140ms/step - loss: 0.2200 - accuracy: 0.9049 - val_loss: 0.2928 - val_accuracy: 0.8577
Epoch 4/50
309/309 [==============================] - 42s 136ms/step - loss: 0.2011 - accuracy: 0.9156 - val_loss: 0.1948 - val_accuracy: 0.9161
Epoch 5/50
309/309 [==============================] - 43s 140ms/step - loss: 0.1949 - accuracy: 0.9183 - val_loss: 0.2347 - val_accuracy: 0.9161
Epoch 6/50
309/309 [==============================] - 43s 139ms/step - loss: 0.1850 - accuracy: 0.9232 - val_loss: 0.3850 - val_accuracy: 0.8642
Epoch 7/50
309/309 [==============================] - 43s 140ms/step - loss: 0.1696 - accuracy: 0.9280 - val_loss: 0.2246 - val_a

In [6]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121, VGG16
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 数据集路径
data_dir = "TrainData_3/TrainData_3"

# 定义数据生成器
datagen = ImageDataGenerator(rescale=1./255)

# 加载数据集
batch_size = 32
image_height = 224
image_width = 224

# 加载landslide数据
landslide_dir = os.path.join(data_dir, "landslide")
landslide_dem_dir = os.path.join(landslide_dir, "dem")
landslide_image_dir = os.path.join(landslide_dir, "image")
landslide_mask_dir = os.path.join(landslide_dir, "mask")

# 加载non-landslide数据
non_landslide_dir = os.path.join(data_dir, "non-landslide")
non_landslide_dem_dir = os.path.join(non_landslide_dir, "dem")
non_landslide_image_dir = os.path.join(non_landslide_dir, "image")

# 读取并预处理数据函数
def read_and_preprocess_data(image_dir, dem_dir=None):
    image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if not filename.startswith('.')]
    images = [tf.keras.preprocessing.image.load_img(img_path, target_size=(image_height, image_width)) for img_path in image_paths]
    images = [tf.keras.preprocessing.image.img_to_array(img) for img in images]
    images = np.array(images)

    if dem_dir:
        dem_paths = [os.path.join(dem_dir, filename) for filename in os.listdir(dem_dir) if not filename.startswith('.')]
        dem_images = [tf.keras.preprocessing.image.load_img(dem_path, target_size=(image_height, image_width)) for dem_path in dem_paths]
        dem_images = [tf.keras.preprocessing.image.img_to_array(dem) for dem in dem_images]
        dem_images = np.array(dem_images)
        return images, dem_images
    else:
        return images

# 读取并预处理landslide数据
landslide_images, landslide_dem_images = read_and_preprocess_data(landslide_image_dir, landslide_dem_dir)
landslide_labels = np.ones(len(landslide_images))

# 读取并预处理non-landslide数据
non_landslide_images = read_and_preprocess_data(non_landslide_image_dir)
non_landslide_labels = np.zeros(len(non_landslide_images))

# 合并数据集
images = np.concatenate((landslide_images, non_landslide_images), axis=0)
dem_images = np.concatenate((landslide_dem_images, np.zeros_like(non_landslide_images)), axis=0)  # 对于non-landslide数据，DEM图像全为0
labels = np.concatenate((landslide_labels, non_landslide_labels), axis=0)

# 切分数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# 构建图像处理模型（使用DenseNet121）
input_image = Input(shape=(image_height, image_width, 3))
base_model_image = DenseNet121(include_top=False, weights='imagenet', input_tensor=input_image)
image_features = GlobalAveragePooling2D()(base_model_image.output)
image_output = Dense(256, activation='relu')(image_features)

# 构建DEM处理模型（使用VGG16）
input_dem = Input(shape=(image_height, image_width, 3))
base_model_dem = VGG16(include_top=False, weights='imagenet', input_tensor=input_dem)
dem_features = GlobalAveragePooling2D()(base_model_dem.output)
dem_output = Dense(256, activation='relu')(dem_features)

# 将两个模型输出连接
merged = Concatenate()([image_output, dem_output])

# 最终的分类层
output = Dense(1, activation='sigmoid')(merged)

# 定义模型
model = Model(inputs=[input_image, input_dem], outputs=output)

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit([X_train, dem_images[:len(X_train)]], y_train, batch_size=batch_size, epochs=50, validation_split=0.2)

# 评估模型
test_loss, test_acc = model.evaluate([X_test, dem_images[len(X_train):]], y_test, verbose=2)
print('\nTest accuracy:', test_acc)

# 模型预测
y_pred = model.predict([X_test, dem_images[len(X_train):]])
y_pred_classes = np.round(y_pred)

# 输出性能报告结果
report = classification_report(y_test, y_pred_classes, target_names=['non-landslide', 'landslide'])
print("Classification Report:\n", report)

# 输出其他性能指标
accuracy = np.mean(y_test == y_pred_classes)
precision = np.mean(y_pred_classes[y_test == 1] == 1)
recall = np.mean(y_pred_classes[y_test == 1] == y_test[y_test == 1])
f1_score = 2 * precision * recall / (precision + recall)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


Epoch 1/50
309/309 [==============================] - 62s 161ms/step - loss: 0.4167 - accuracy: 0.8924 - val_loss: 0.6544 - val_accuracy: 0.7868
Epoch 2/50
309/309 [==============================] - 48s 155ms/step - loss: 0.2151 - accuracy: 0.9101 - val_loss: 0.1937 - val_accuracy: 0.9149
Epoch 3/50
309/309 [==============================] - 45s 144ms/step - loss: 0.1951 - accuracy: 0.9199 - val_loss: 0.4133 - val_accuracy: 0.8610
Epoch 4/50
309/309 [==============================] - 47s 152ms/step - loss: 0.1886 - accuracy: 0.9195 - val_loss: 0.2489 - val_accuracy: 0.8954
Epoch 5/50
309/309 [==============================] - 45s 147ms/step - loss: 0.1837 - accuracy: 0.9229 - val_loss: 0.4556 - val_accuracy: 0.8168
Epoch 6/50
309/309 [==============================] - 46s 150ms/step - loss: 0.1712 - accuracy: 0.9302 - val_loss: 0.2466 - val_accuracy: 0.9035
Epoch 7/50
309/309 [==============================] - 45s 147ms/step - loss: 0.1634 - accuracy: 0.9304 - val_loss: 0.1922 - val_ac

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19, MobileNetV2
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 数据集路径
data_dir = "TrainData_3/TrainData_3"

# 定义数据生成器
datagen = ImageDataGenerator(rescale=1./255)

# 加载数据集
batch_size = 8
image_height = 224
image_width = 224

# 加载landslide数据
landslide_dir = os.path.join(data_dir, "landslide")
landslide_dem_dir = os.path.join(landslide_dir, "dem")
landslide_image_dir = os.path.join(landslide_dir, "image")
landslide_mask_dir = os.path.join(landslide_dir, "mask")

# 加载non-landslide数据
non_landslide_dir = os.path.join(data_dir, "non-landslide")
non_landslide_dem_dir = os.path.join(non_landslide_dir, "dem")
non_landslide_image_dir = os.path.join(non_landslide_dir, "image")

# 读取并预处理数据函数
def read_and_preprocess_data(image_dir, dem_dir=None):
    image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if not filename.startswith('.')]
    images = [tf.keras.preprocessing.image.load_img(img_path, target_size=(image_height, image_width)) for img_path in image_paths]
    images = [tf.keras.preprocessing.image.img_to_array(img) for img in images]
    images = np.array(images)

    if dem_dir:
        dem_paths = [os.path.join(dem_dir, filename) for filename in os.listdir(dem_dir) if not filename.startswith('.')]
        dem_images = [tf.keras.preprocessing.image.load_img(dem_path, target_size=(image_height, image_width)) for dem_path in dem_paths]
        dem_images = [tf.keras.preprocessing.image.img_to_array(dem) for dem in dem_images]
        dem_images = np.array(dem_images)
        return images, dem_images
    else:
        return images

# 读取并预处理landslide数据
landslide_images, landslide_dem_images = read_and_preprocess_data(landslide_image_dir, landslide_dem_dir)
landslide_labels = np.ones(len(landslide_images))

# 读取并预处理non-landslide数据
non_landslide_images = read_and_preprocess_data(non_landslide_image_dir)
non_landslide_labels = np.zeros(len(non_landslide_images))

# 合并数据集
images = np.concatenate((landslide_images, non_landslide_images), axis=0)
dem_images = np.concatenate((landslide_dem_images, np.zeros_like(non_landslide_images)), axis=0)  # 对于non-landslide数据，DEM图像全为0
labels = np.concatenate((landslide_labels, non_landslide_labels), axis=0)

# 切分数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# 构建图像处理模型（使用VGG19）
input_image = Input(shape=(image_height, image_width, 3))
base_model_image = VGG19(include_top=False, weights='imagenet', input_tensor=input_image)
image_features = GlobalAveragePooling2D()(base_model_image.output)
image_output = Dense(256, activation='relu')(image_features)

# 构建DEM处理模型（使用MobileNetV2）
input_dem = Input(shape=(image_height, image_width, 3))
base_model_dem = MobileNetV2(include_top=False, weights='imagenet', input_tensor=input_dem)
dem_features = GlobalAveragePooling2D()(base_model_dem.output)
dem_output = Dense(256, activation='relu')(dem_features)

# 将两个模型输出连接
merged = Concatenate()([image_output, dem_output])

# 最终的分类层
output = Dense(1, activation='sigmoid')(merged)

# 定义模型
model = Model(inputs=[input_image, input_dem], outputs=output)

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit([X_train, dem_images[:len(X_train)]], y_train, batch_size=batch_size, epochs=50, validation_split=0.2)

# 评估模型
test_loss, test_acc = model.evaluate([X_test, dem_images[len(X_train):]], y_test, verbose=2)
print('\nTest accuracy:', test_acc)

# 模型预测
y_pred = model.predict([X_test, dem_images[len(X_train):]])
y_pred_classes = np.round(y_pred)

# 输出性能报告结果
report = classification_report(y_test, y_pred_classes, target_names=['non-landslide', 'landslide'])
print("Classification Report:\n", report)

# 输出其他性能指标
accuracy = np.mean(y_test == y_pred_classes)
precision = np.mean(y_pred_classes[y_test == 1] == 1)
recall = np.mean(y_pred_classes[y_test == 1] == y_test[y_test == 1])
f1_score = 2 * precision * recall / (precision + recall)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


2024-05-20 02:40:36.995110: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-20 02:41:13.391603: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-20 02:41:13.758914: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46662 MB memory:  -> device: 0, name: NVIDIA L20, pci bus id: 0000:71:01.0, compute capability: 8.9


Epoch 1/50


2024-05-20 02:41:29.105987: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101
2024-05-20 02:41:29.822853: W tensorflow/stream_executor/gpu/asm_compiler.cc:230] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.9
2024-05-20 02:41:29.822872: W tensorflow/stream_executor/gpu/asm_compiler.cc:233] Used ptxas at ptxas
2024-05-20 02:41:29.822916: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2024-05-20 02:41:30.605609: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1234/1234 [==============================] - 81s 60ms/step - loss: 0.9418 - accuracy: 0.7464 - val_loss: 0.6020 - val_accuracy: 0.7175
Epoch 2/50
1234/1234 [==============================] - 70s 57ms/step - loss: 0.5969 - accuracy: 0.7198 - val_loss: 0.5988 - val_accuracy: 0.7175
Epoch 3/50
1234/1234 [==============================] - 69s 56ms/step - loss: 0.6013 - accuracy: 0.7190 - val_loss: 0.5956 - val_accuracy: 0.7175
Epoch 4/50
1234/1234 [==============================] - 69s 56ms/step - loss: 0.5953 - accuracy: 0.7198 - val_loss: 0.5956 - val_accuracy: 0.7175
Epoch 5/50
1234/1234 [==============================] - 68s 55ms/step - loss: 0.5951 - accuracy: 0.7198 - val_loss: 0.5964 - val_accuracy: 0.7175
Epoch 6/50
1234/1234 [==============================] - 68s 55ms/step - loss: 0.5946 - accuracy: 0.7198 - val_loss: 0.5955 - val_accuracy: 0.7175
Epoch 7/50
1234/1234 [==============================] - 69s 56ms/step - loss: 0.5971 - accuracy: 0.7196 - val_loss: 0.5962 - val_accura

/root/miniconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/miniconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/miniconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/tmp/ipykernel_1206/799019489.py:108: RuntimeWarn

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3, DenseNet121
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 数据集路径
data_dir = "TrainData_3/TrainData_3"

# 定义数据生成器
datagen = ImageDataGenerator(rescale=1./255)

# 加载数据集
batch_size = 32
image_height = 224
image_width = 224

# 加载landslide数据
landslide_dir = os.path.join(data_dir, "landslide")
landslide_dem_dir = os.path.join(landslide_dir, "dem")
landslide_image_dir = os.path.join(landslide_dir, "image")
landslide_mask_dir = os.path.join(landslide_dir, "mask")

# 加载non-landslide数据
non_landslide_dir = os.path.join(data_dir, "non-landslide")
non_landslide_dem_dir = os.path.join(non_landslide_dir, "dem")
non_landslide_image_dir = os.path.join(non_landslide_dir, "image")

# 读取并预处理数据函数
def read_and_preprocess_data(image_dir, dem_dir=None):
    image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if not filename.startswith('.')]
    images = [tf.keras.preprocessing.image.load_img(img_path, target_size=(image_height, image_width)) for img_path in image_paths]
    images = [tf.keras.preprocessing.image.img_to_array(img) for img in images]
    images = np.array(images)

    if dem_dir:
        dem_paths = [os.path.join(dem_dir, filename) for filename in os.listdir(dem_dir) if not filename.startswith('.')]
        dem_images = [tf.keras.preprocessing.image.load_img(dem_path, target_size=(image_height, image_width)) for dem_path in dem_paths]
        dem_images = [tf.keras.preprocessing.image.img_to_array(dem) for dem in dem_images]
        dem_images = np.array(dem_images)
        return images, dem_images
    else:
        return images

# 读取并预处理landslide数据
landslide_images, landslide_dem_images = read_and_preprocess_data(landslide_image_dir, landslide_dem_dir)
landslide_labels = np.ones(len(landslide_images))

# 读取并预处理non-landslide数据
non_landslide_images = read_and_preprocess_data(non_landslide_image_dir)
non_landslide_labels = np.zeros(len(non_landslide_images))

# 合并数据集
images = np.concatenate((landslide_images, non_landslide_images), axis=0)
dem_images = np.concatenate((landslide_dem_images, np.zeros_like(non_landslide_images)), axis=0)  # 对于non-landslide数据，DEM图像全为0
labels = np.concatenate((landslide_labels, non_landslide_labels), axis=0)

# 切分数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# 构建图像处理模型（使用InceptionV3）
input_image = Input(shape=(image_height, image_width, 3))
base_model_image = InceptionV3(include_top=False, weights='imagenet', input_tensor=input_image)
image_features = GlobalAveragePooling2D()(base_model_image.output)
image_output = Dense(256, activation='relu')(image_features)

# 构建DEM处理模型（使用DenseNet121）
input_dem = Input(shape=(image_height, image_width, 3))
base_model_dem = DenseNet121(include_top=False, weights='imagenet', input_tensor=input_dem)
dem_features = GlobalAveragePooling2D()(base_model_dem.output)
dem_output = Dense(256, activation='relu')(dem_features)

# 将两个模型输出连接
merged = Concatenate()([image_output, dem_output])

# 最终的分类层
output = Dense(1, activation='sigmoid')(merged)

# 定义模型
model = Model(inputs=[input_image, input_dem], outputs=output)

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit([X_train, dem_images[:len(X_train)]], y_train, batch_size=batch_size, epochs=50, validation_split=0.2)

# 评估模型
test_loss, test_acc = model.evaluate([X_test, dem_images[len(X_train):]], y_test, verbose=2)
print('\nTest accuracy:', test_acc)

# 模型预测
y_pred = model.predict([X_test, dem_images[len(X_train):]])
y_pred_classes = np.round(y_pred)

# 输出性能报告结果
report = classification_report(y_test, y_pred_classes, target_names=['non-landslide', 'landslide'])
print("Classification Report:\n", report)

# 输出其他性能指标
accuracy = np.mean(y_test == y_pred_classes)
precision = np.mean(y_pred_classes[y_test == 1] == 1)
recall = np.mean(y_pred_classes[y_test == 1] == y_test[y_test == 1])
f1_score = 2 * precision * recall / (precision + recall)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


2024-05-19 21:48:15.968806: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-19 21:48:52.508397: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-19 21:48:52.970311: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46662 MB memory:  -> device: 0, name: NVIDIA L20, pci bus id: 0000:71:01.0, compute capability: 8.9


Epoch 1/50


2024-05-19 21:49:22.225603: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101
2024-05-19 21:49:22.958132: W tensorflow/stream_executor/gpu/asm_compiler.cc:230] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.9
2024-05-19 21:49:22.958157: W tensorflow/stream_executor/gpu/asm_compiler.cc:233] Used ptxas at ptxas
2024-05-19 21:49:22.958221: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2024-05-19 21:49:24.126855: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


309/309 [==============================] - 63s 146ms/step - loss: 0.2906 - accuracy: 0.8778 - val_loss: 0.4110 - val_accuracy: 0.8557
Epoch 2/50
309/309 [==============================] - 41s 133ms/step - loss: 0.2326 - accuracy: 0.9019 - val_loss: 0.2287 - val_accuracy: 0.8970
Epoch 3/50
309/309 [==============================] - 41s 134ms/step - loss: 0.2164 - accuracy: 0.9095 - val_loss: 0.2298 - val_accuracy: 0.9189
Epoch 4/50
309/309 [==============================] - 40s 130ms/step - loss: 0.1872 - accuracy: 0.9199 - val_loss: 0.4527 - val_accuracy: 0.7900
Epoch 5/50
309/309 [==============================] - 39s 125ms/step - loss: 0.1812 - accuracy: 0.9252 - val_loss: 0.2151 - val_accuracy: 0.9181
Epoch 6/50
309/309 [==============================] - 40s 128ms/step - loss: 0.1724 - accuracy: 0.9307 - val_loss: 0.2101 - val_accuracy: 0.9043
Epoch 7/50
309/309 [==============================] - 40s 128ms/step - loss: 0.1683 - accuracy: 0.9315 - val_loss: 0.3464 - val_accuracy: 0.8

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2, Xception
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 数据集路径
data_dir = "TrainData_3/TrainData_3"

# 定义数据生成器
datagen = ImageDataGenerator(rescale=1./255)

# 加载数据集
batch_size = 32
image_height = 224
image_width = 224

# 加载landslide数据
landslide_dir = os.path.join(data_dir, "landslide")
landslide_dem_dir = os.path.join(landslide_dir, "dem")
landslide_image_dir = os.path.join(landslide_dir, "image")
landslide_mask_dir = os.path.join(landslide_dir, "mask")

# 加载non-landslide数据
non_landslide_dir = os.path.join(data_dir, "non-landslide")
non_landslide_dem_dir = os.path.join(non_landslide_dir, "dem")
non_landslide_image_dir = os.path.join(non_landslide_dir, "image")

# 读取并预处理数据函数
def read_and_preprocess_data(image_dir, dem_dir=None):
    image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if not filename.startswith('.')]
    images = [tf.keras.preprocessing.image.load_img(img_path, target_size=(image_height, image_width)) for img_path in image_paths]
    images = [tf.keras.preprocessing.image.img_to_array(img) for img in images]
    images = np.array(images)

    if dem_dir:
        dem_paths = [os.path.join(dem_dir, filename) for filename in os.listdir(dem_dir) if not filename.startswith('.')]
        dem_images = [tf.keras.preprocessing.image.load_img(dem_path, target_size=(image_height, image_width)) for dem_path in dem_paths]
        dem_images = [tf.keras.preprocessing.image.img_to_array(dem) for dem in dem_images]
        dem_images = np.array(dem_images)
        return images, dem_images
    else:
        return images

# 读取并预处理landslide数据
landslide_images, landslide_dem_images = read_and_preprocess_data(landslide_image_dir, landslide_dem_dir)
landslide_labels = np.ones(len(landslide_images))

# 读取并预处理non-landslide数据
non_landslide_images = read_and_preprocess_data(non_landslide_image_dir)
non_landslide_labels = np.zeros(len(non_landslide_images))

# 合并数据集
images = np.concatenate((landslide_images, non_landslide_images), axis=0)
dem_images = np.concatenate((landslide_dem_images, np.zeros_like(non_landslide_images)), axis=0)  # 对于non-landslide数据，DEM图像全为0
labels = np.concatenate((landslide_labels, non_landslide_labels), axis=0)

# 切分数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# 构建图像处理模型（使用MobileNetV2）
input_image = Input(shape=(image_height, image_width, 3))
base_model_image = MobileNetV2(include_top=False, weights='imagenet', input_tensor=input_image)
image_features = GlobalAveragePooling2D()(base_model_image.output)
image_output = Dense(256, activation='relu')(image_features)

# 构建DEM处理模型（使用Xception）
input_dem = Input(shape=(image_height, image_width, 3))
base_model_dem = Xception(include_top=False, weights='imagenet', input_tensor=input_dem)
dem_features = GlobalAveragePooling2D()(base_model_dem.output)
dem_output = Dense(256, activation='relu')(dem_features)

# 将两个模型输出连接
merged = Concatenate()([image_output, dem_output])

# 最终的分类层
output = Dense(1, activation='sigmoid')(merged)

# 定义模型
model = Model(inputs=[input_image, input_dem], outputs=output)

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit([X_train, dem_images[:len(X_train)]], y_train, batch_size=batch_size, epochs=50, validation_split=0.2)

# 评估模型
test_loss, test_acc = model.evaluate([X_test, dem_images[len(X_train):]], y_test, verbose=2)
print('\nTest accuracy:', test_acc)

# 模型预测
y_pred = model.predict([X_test, dem_images[len(X_train):]])
y_pred_classes = np.round(y_pred)

# 输出性能报告结果
report = classification_report(y_test, y_pred_classes, target_names=['non-landslide', 'landslide'])
print("Classification Report:\n", report)

# 输出其他性能指标
accuracy = np.mean(y_test == y_pred_classes)
precision = np.mean(y_pred_classes[y_test == 1] == 1)
recall = np.mean(y_pred_classes[y_test == 1] == y_test[y_test == 1])
f1_score = 2 * precision * recall / (precision + recall)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


Epoch 1/50
309/309 [==============================] - 91s 266ms/step - loss: 0.2561 - accuracy: 0.8939 - val_loss: 1.5745 - val_accuracy: 0.6437
Epoch 2/50
309/309 [==============================] - 78s 253ms/step - loss: 0.2111 - accuracy: 0.9120 - val_loss: 3.2450 - val_accuracy: 0.4422
Epoch 3/50
309/309 [==============================] - 78s 254ms/step - loss: 0.1837 - accuracy: 0.9226 - val_loss: 2.7858 - val_accuracy: 0.5351
Epoch 4/50
309/309 [==============================] - 78s 253ms/step - loss: 0.1681 - accuracy: 0.9304 - val_loss: 2.3337 - val_accuracy: 0.5882
Epoch 5/50
309/309 [==============================] - 78s 253ms/step - loss: 0.1517 - accuracy: 0.9397 - val_loss: 5.2514 - val_accuracy: 0.3470
Epoch 6/50
309/309 [==============================] - 77s 251ms/step - loss: 0.1378 - accuracy: 0.9443 - val_loss: 0.6163 - val_accuracy: 0.8087
Epoch 7/50
309/309 [==============================] - 78s 254ms/step - loss: 0.1272 - accuracy: 0.9511 - val_loss: 6.2211 - val_ac

In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0, ResNet152V2
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 数据集路径
data_dir = "TrainData_3/TrainData_3"

# 定义数据生成器
datagen = ImageDataGenerator(rescale=1./255)

# 加载数据集
batch_size = 32
image_height = 224
image_width = 224

# 加载landslide数据
landslide_dir = os.path.join(data_dir, "landslide")
landslide_dem_dir = os.path.join(landslide_dir, "dem")
landslide_image_dir = os.path.join(landslide_dir, "image")
landslide_mask_dir = os.path.join(landslide_dir, "mask")

# 加载non-landslide数据
non_landslide_dir = os.path.join(data_dir, "non-landslide")
non_landslide_dem_dir = os.path.join(non_landslide_dir, "dem")
non_landslide_image_dir = os.path.join(non_landslide_dir, "image")

# 读取并预处理数据函数
def read_and_preprocess_data(image_dir, dem_dir=None):
    image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if not filename.startswith('.')]
    images = [tf.keras.preprocessing.image.load_img(img_path, target_size=(image_height, image_width)) for img_path in image_paths]
    images = [tf.keras.preprocessing.image.img_to_array(img) for img in images]
    images = np.array(images)

    if dem_dir:
        dem_paths = [os.path.join(dem_dir, filename) for filename in os.listdir(dem_dir) if not filename.startswith('.')]
        dem_images = [tf.keras.preprocessing.image.load_img(dem_path, target_size=(image_height, image_width)) for dem_path in dem_paths]
        dem_images = [tf.keras.preprocessing.image.img_to_array(dem) for dem in dem_images]
        dem_images = np.array(dem_images)
        return images, dem_images
    else:
        return images

# 读取并预处理landslide数据
landslide_images, landslide_dem_images = read_and_preprocess_data(landslide_image_dir, landslide_dem_dir)
landslide_labels = np.ones(len(landslide_images))

# 读取并预处理non-landslide数据
non_landslide_images = read_and_preprocess_data(non_landslide_image_dir)
non_landslide_labels = np.zeros(len(non_landslide_images))

# 合并数据集
images = np.concatenate((landslide_images, non_landslide_images), axis=0)
dem_images = np.concatenate((landslide_dem_images, np.zeros_like(non_landslide_images)), axis=0)  # 对于non-landslide数据，DEM图像全为0
labels = np.concatenate((landslide_labels, non_landslide_labels), axis=0)

# 切分数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# 构建图像处理模型（使用EfficientNetB0）
input_image = Input(shape=(image_height, image_width, 3))
base_model_image = EfficientNetB0(include_top=False, weights='imagenet', input_tensor=input_image)
image_features = GlobalAveragePooling2D()(base_model_image.output)
image_output = Dense(256, activation='relu')(image_features)

# 构建DEM处理模型（使用ResNet152V2）
input_dem = Input(shape=(image_height, image_width, 3))
base_model_dem = ResNet152V2(include_top=False, weights='imagenet', input_tensor=input_dem)
dem_features = GlobalAveragePooling2D()(base_model_dem.output)
dem_output = Dense(256, activation='relu')(dem_features)

# 将两个模型输出连接
merged = Concatenate()([image_output, dem_output])

# 最终的分类层
output = Dense(1, activation='sigmoid')(merged)

# 定义模型
model = Model(inputs=[input_image, input_dem], outputs=output)

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit([X_train, dem_images[:len(X_train)]], y_train, batch_size=batch_size, epochs=50, validation_split=0.2)

# 评估模型
test_loss, test_acc = model.evaluate([X_test, dem_images[len(X_train):]], y_test, verbose=2)
print('\nTest accuracy:', test_acc)

# 模型预测
y_pred = model.predict([X_test, dem_images[len(X_train):]])
y_pred_classes = np.round(y_pred)

# 输出性能报告结果
report = classification_report(y_test, y_pred_classes, target_names=['non-landslide', 'landslide'])
print("Classification Report:\n", report)

# 输出其他性能指标
accuracy = np.mean(y_test == y_pred_classes)
precision = np.mean(y_pred_classes[y_test == 1] == 1)
recall = np.mean(y_pred_classes[y_test == 1] == y_test[y_test == 1])
f1_score = 2 * precision * recall / (precision + recall)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


234545216/234545216 [==============================] - 930s 4us/step
Epoch 1/50
309/309 [==============================] - 96s 249ms/step - loss: 0.2349 - accuracy: 0.9024 - val_loss: 0.2595 - val_accuracy: 0.9039
Epoch 2/50
309/309 [==============================] - 71s 229ms/step - loss: 0.1666 - accuracy: 0.9358 - val_loss: 0.4231 - val_accuracy: 0.8091
Epoch 3/50
309/309 [==============================] - 70s 228ms/step - loss: 0.1419 - accuracy: 0.9431 - val_loss: 0.1871 - val_accuracy: 0.9238
Epoch 4/50
309/309 [==============================] - 71s 229ms/step - loss: 0.1157 - accuracy: 0.9516 - val_loss: 0.2318 - val_accuracy: 0.9218
Epoch 5/50
309/309 [==============================] - 72s 234ms/step - loss: 0.1054 - accuracy: 0.9593 - val_loss: 0.1794 - val_accuracy: 0.9343
Epoch 6/50
309/309 [==============================] - 71s 231ms/step - loss: 0.0837 - accuracy: 0.9661 - val_loss: 784282905739264.0000 - val_accuracy: 0.7175
Epoch 7/50
309/309 [===========================

In [4]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import NASNetMobile, Xception
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 数据集路径
data_dir = "TrainData_3/TrainData_3"

# 定义数据生成器
datagen = ImageDataGenerator(rescale=1./255)

# 加载数据集
batch_size = 32
image_height = 224
image_width = 224

# 加载landslide数据
landslide_dir = os.path.join(data_dir, "landslide")
landslide_dem_dir = os.path.join(landslide_dir, "dem")
landslide_image_dir = os.path.join(landslide_dir, "image")
landslide_mask_dir = os.path.join(landslide_dir, "mask")

# 加载non-landslide数据
non_landslide_dir = os.path.join(data_dir, "non-landslide")
non_landslide_dem_dir = os.path.join(non_landslide_dir, "dem")
non_landslide_image_dir = os.path.join(non_landslide_dir, "image")

# 读取并预处理数据函数
def read_and_preprocess_data(image_dir, dem_dir=None):
    image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if not filename.startswith('.')]
    images = [tf.keras.preprocessing.image.load_img(img_path, target_size=(image_height, image_width)) for img_path in image_paths]
    images = [tf.keras.preprocessing.image.img_to_array(img) for img in images]
    images = np.array(images)

    if dem_dir:
        dem_paths = [os.path.join(dem_dir, filename) for filename in os.listdir(dem_dir) if not filename.startswith('.')]
        dem_images = [tf.keras.preprocessing.image.load_img(dem_path, target_size=(image_height, image_width)) for dem_path in dem_paths]
        dem_images = [tf.keras.preprocessing.image.img_to_array(dem) for dem in dem_images]
        dem_images = np.array(dem_images)
        return images, dem_images
    else:
        return images

# 读取并预处理landslide数据
landslide_images, landslide_dem_images = read_and_preprocess_data(landslide_image_dir, landslide_dem_dir)
landslide_labels = np.ones(len(landslide_images))

# 读取并预处理non-landslide数据
non_landslide_images = read_and_preprocess_data(non_landslide_image_dir)
non_landslide_labels = np.zeros(len(non_landslide_images))

# 合并数据集
images = np.concatenate((landslide_images, non_landslide_images), axis=0)
dem_images = np.concatenate((landslide_dem_images, np.zeros_like(non_landslide_images)), axis=0)  # 对于non-landslide数据，DEM图像全为0
labels = np.concatenate((landslide_labels, non_landslide_labels), axis=0)

# 切分数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# 构建图像处理模型（使用NASNetMobile）
input_image = Input(shape=(image_height, image_width, 3))
base_model_image = NASNetMobile(include_top=False, weights='imagenet', input_tensor=input_image)
image_features = GlobalAveragePooling2D()(base_model_image.output)
image_output = Dense(256, activation='relu')(image_features)

# 构建DEM处理模型（使用Xception）
input_dem = Input(shape=(image_height, image_width, 3))
base_model_dem = Xception(include_top=False, weights='imagenet', input_tensor=input_dem)
dem_features = GlobalAveragePooling2D()(base_model_dem.output)
dem_output = Dense(256, activation='relu')(dem_features)

# 将两个模型输出连接
merged = Concatenate()([image_output, dem_output])

# 最终的分类层
output = Dense(1, activation='sigmoid')(merged)

# 定义模型
model = Model(inputs=[input_image, input_dem], outputs=output)

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit([X_train, dem_images[:len(X_train)]], y_train, batch_size=batch_size, epochs=50, validation_split=0.2)

# 评估模型
test_loss, test_acc = model.evaluate([X_test, dem_images[len(X_train):]], y_test, verbose=2)
print('\nTest accuracy:', test_acc)

# 模型预测
y_pred = model.predict([X_test, dem_images[len(X_train):]])
y_pred_classes = np.round(y_pred)

# 输出性能报告结果
report = classification_report(y_test, y_pred_classes, target_names=['non-landslide', 'landslide'])
print("Classification Report:\n", report)

# 输出其他性能指标
accuracy = np.mean(y_test == y_pred_classes)
precision = np.mean(y_pred_classes[y_test == 1] == 1)
recall = np.mean(y_pred_classes[y_test == 1] == y_test[y_test == 1])
f1_score = 2 * precision * recall / (precision + recall)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


19993432/19993432 [==============================] - 2s 0us/step
Epoch 1/50
309/309 [==============================] - 123s 335ms/step - loss: 0.2247 - accuracy: 0.9071 - val_loss: 43.0701 - val_accuracy: 0.7175
Epoch 2/50
309/309 [==============================] - 100s 323ms/step - loss: 0.1625 - accuracy: 0.9332 - val_loss: 10.1262 - val_accuracy: 0.7061
Epoch 3/50
309/309 [==============================] - 101s 327ms/step - loss: 0.1442 - accuracy: 0.9411 - val_loss: 3.9546 - val_accuracy: 0.8468
Epoch 4/50
309/309 [==============================] - 100s 323ms/step - loss: 0.1119 - accuracy: 0.9562 - val_loss: 8.4028 - val_accuracy: 0.8537
Epoch 5/50
309/309 [==============================] - 99s 322ms/step - loss: 0.0892 - accuracy: 0.9654 - val_loss: 17.8243 - val_accuracy: 0.7179
Epoch 6/50
309/309 [==============================] - 99s 322ms/step - loss: 0.0937 - accuracy: 0.9670 - val_loss: 0.7768 - val_accuracy: 0.7884
Epoch 7/50
309/309 [==============================] - 100s

# 分割